In [ ]:
--Salesssss--

SELECT sod.SalesOrderID,
        SalesOrderDetailID,
        OrderQty,
        sod. ProductID,
        p.Name 'Product',
        UnitPrice,
        LineTotal,
        st.Name 'Region',
        soh.OrderDate,
        soh.OnlineOrderFlag,
        ps.Name 'Subcategory'
from Sales.SalesOrderDetail sod 
JOIN sales.SalesOrderHeader soh 
    on soh.SalesOrderID = sod.SalesOrderID
JOIN Sales.SalesTerritory st 
    on soh.TerritoryID = st.TerritoryID
JOIN Production.Product p 
    on p.ProductID = sod.ProductID
join Production.ProductSubcategory ps ON
    ps.ProductSubcategoryID = p.ProductSubcategoryID


In [ ]:
--Warehouse--

SELECT pi.*, l.Name 'Warehouse',
l.LocationID, 
pc.Name 'Category_Name', 
p.Name 'Product_Name', 
p.SafetyStockLevel, 
p.StandardCost, 
p.ListPrice,
case
when pi.Quantity > p.SafetyStockLevel 
    then (pi.Quantity - p.SafetyStockLevel) * p.ListPrice
    else 0
    end as 'Predicted_Sales',
case 
when pi.Quantity < p.SafetyStockLevel
    then (p.SafetyStockLevel - pi.Quantity) * p.StandardCost
    else 0
    end as 'Predicted_Costs'
FROM Production.ProductInventory pi 
JOIN Production.[Location] l 
    on l.LocationID = pi.LocationID
JOIN Production.Product p 
    on p.ProductID = pi.ProductID
JOIN Production.ProductCategory pc 
    on pc.ProductCategoryID = p.ProductSubcategoryID




In [ ]:
--Clients--

select a.AddressID,
    a.AddressLine1,
    a.AddressLine2,
    cr.Name 'Country',
    a.City,
    sp.Name 'StateProvince',
    a.PostalCode,
    bea.BusinessEntityID,
    at.Name 'AddressType',
    ct.Name 'ContactType',
    sp.TerritoryID
from Person.Address a
join Person.BusinessEntityAddress bea 
    on bea.AddressID = a.AddressID
JOIN Person.AddressType at 
    on at.AddressTypeID = bea.AddressTypeID
join Person.BusinessEntityContact bec 
    on bec.BusinessEntityID = bea.BusinessEntityID
join Person.ContactType ct 
    on ct.ContactTypeID = bec.ContactTypeID
join Person.StateProvince sp 
    on sp.StateProvinceID = a.StateProvinceID
JOIN Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
JOIN Sales.Customer c 
    on c.CustomerID = bea.BusinessEntityID


In [ ]:
--Customer 2-- plik

SELECT s.BusinessEntityID 'CustomerID'
    ,s.Name 'CustomerName'
    ,a.AddressLine1
    ,a.City
    ,sp.Name 'StateProvince'
    ,cr.Name 'Country'
    ,soh.TotalDue
    ,'Retail' 'CustomerType'
FROM Sales.Store s
JOIN Sales.Customer c 
    on c.StoreID = s.BusinessEntityID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = s.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
JOIN Person.StateProvince sp 
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
JOIN Sales.SalesOrderHeader soh 
    on soh.CustomerID = c.CustomerID
union all 
SELECT c.CustomerID
    ,CONCAT_WS(' ',p.FirstName,p.LastName) 'CustomerName'
    ,a.AddressLine1
    ,a.City
    ,sp.Name 'StateProvince'
    ,cr.Name 'Country'
    ,soh.TotalDue
    ,'Individual' 'CustomerType'
FROM Sales.Customer c
JOIN Person.Person p
    on p.BusinessEntityID = c.PersonID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = p.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
JOIN Person.StateProvince sp 
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
JOIN Sales.SalesOrderHeader soh 
    on soh.CustomerID = c.CustomerID



In [ ]:
--Production losses--

SELECT p.ProductID,
        p.Name,
        p.SafetyStockLevel,
        p.ReorderPoint,
        p.StandardCost,
        p.ListPrice,
        p.DaysToManufacture,
        p.ProductLine,
        pc.Name 'Category',
        ps.Name 'Subcategory',
        wo.ScrappedQty,
        wo.ScrappedQty * p.StandardCost 'ProductionLosses',
        wo.ScrappedQty * p.ListPrice 'LoosedIncomes',
case 
    when p.ProductLine is null
    then 'L'
    else p.ProductLine
    end as ProductLine2 
from Production.[Product] p 
join Production.ProductSubcategory ps ON
    ps.ProductSubcategoryID = p.ProductSubcategoryID
join Production.WorkOrder wo ON
    wo.ProductID = p.ProductID
join Production.ProductCategory pc ON
    pc.ProductCategoryID = ps.ProductCategoryID




In [ ]:
--Employees--

select p.BusinessEntityID
    ,p.PersonType
    ,CONCAT_WS(' ',p.FirstName,p.MiddleName,p.LastName) Employee
    ,e.Gender
    ,d.Name Department
    ,e.JobTitle
    ,pp.PhoneNumber
    ,a.City
    ,CONCAT_WS(' ',a.AddressLine1,a.AddressLine2) Address
    ,sp.Name StateProvince 
    ,cr.Name Country
from Person.Person p 
JOIN HumanResources.EmployeeDepartmentHistory edh 
    on edh.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.Department d 
    on d.DepartmentID = edh.DepartmentID
join HumanResources.Employee e 
    on e.BusinessEntityID = p.BusinessEntityID
join Person.PersonPhone pp
    on pp.BusinessEntityID = p.BusinessEntityID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = p.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
join Person.StateProvince sp
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
order by p.BusinessEntityID

In [ ]:
--SalesPerson and sales by Categories--

SELECT CONCAT_WS(' ', p.FirstName,p.LastName) SalesPerson
    ,REPLACE(case when st.name is null then 'Online' else st.name end ,'null','Online') Region
    ,sum(soh.TotalDue) TotalSales
    ,pc.Name CategoryName
from Sales.SalesPerson sp
join Person.Person p 
    on p.BusinessEntityID = sp.BusinessEntityID
left JOIN Sales.SalesTerritory st 
    on sp.TerritoryID = st.TerritoryID
join sales.SalesOrderHeader soh 
    on soh.SalesPersonID = sp.BusinessEntityID
join sales.SalesOrderDetail sod 
    on sod.SalesOrderID = soh.SalesOrderID
join Production.Product pr 
    on pr.ProductID = sod.ProductID
join Production.ProductCategory pc 
    on pc.ProductCategoryID = pr.ProductSubcategoryID
group by sp.BusinessEntityID
    ,p.FirstName
    ,p.LastName
    ,st.Name
    ,pc.Name

In [ ]:
--Top 3 and Last 3 Sales Person--

with  TOP3 as (
SELECT  top 3 sp.BusinessEntityID
    ,CONCAT_WS(' ',p.FirstName,p.LastName) 'SalesPerson'
    ,sp.SalesYTD 'Sales'
    ,'TOP3' 'Rank'
from sales.SalesPerson sp
join Person.Person p 
    on p.BusinessEntityID = sp.BusinessEntityID
order by 'Sales' DESC),
LAST3 as (
SELECT  top 3 sp.BusinessEntityID
    ,CONCAT_WS(' ',p.FirstName,p.LastName) 'SalesPerson'
    ,sp.SalesYTD 'Sales'
    ,'LAST3' 'Rank'
from sales.SalesPerson sp
join Person.Person p 
    on p.BusinessEntityID = sp.BusinessEntityID
order by 'Sales' ASC)


SELECT * 
from TOP3
UNION all 
SELECT * 
FROM LAST3

In [ ]:
--Customers--

SELECT c.CustomerID
    ,CONCAT_WS(' ',p.FirstName,p.LastName) 'CustomerName'
    ,a.AddressLine1
    ,a.City
    ,sp.Name 'StateProvince'
    ,cr.Name 'Country'
    ,soh.TotalDue
    ,'Individual' 'CustomerType'
FROM Sales.Customer c
JOIN Person.Person p
    on p.BusinessEntityID = c.PersonID
JOIN Person.BusinessEntityAddress bea 
    on bea.BusinessEntityID = p.BusinessEntityID
join Person.Address a 
    on a.AddressID = bea.AddressID
JOIN Person.StateProvince sp 
    on sp.StateProvinceID = a.StateProvinceID
join Person.CountryRegion cr 
    on cr.CountryRegionCode = sp.CountryRegionCode
JOIN Sales.SalesOrderHeader soh 
    on soh.CustomerID = c.CustomerID


In [ ]:
--Top 10 products by Region--

SELECT * 
from
(SELECT 
        sod. ProductID,
        p.Name 'Product',
        sum(LineTotal) 'LineTotal1',
        st.Name 'Region',
        ps.Name 'Subcategory',
        row_number() over (PARTITION by st.Name order by sum(LineTotal) DESC) as Rank
from Sales.SalesOrderDetail sod 
JOIN sales.SalesOrderHeader soh 
    on soh.SalesOrderID = sod.SalesOrderID
JOIN Sales.SalesTerritory st 
    on soh.TerritoryID = st.TerritoryID
JOIN Production.Product p 
    on p.ProductID = sod.ProductID
join Production.ProductSubcategory ps ON
    ps.ProductSubcategoryID = p.ProductSubcategoryID
group by 
        sod. ProductID,
        p.Name,
        st.Name ,
        ps.Name,
        LineTotal) ranked
WHERE Rank <=10

In [ ]:
--ostatnie zamówienie na dany region--

SELECT soh.TerritoryID
    ,soh.SalesOrderID
    ,soh.OrderDate
    ,soh.SalesOrderNumber
    ,soh.TotalDue
    ,LAST_VALUE(soh.OrderDate) OVER (PARTITION  by soh.TerritoryID order by soh.SalesOrderID desc)
FROM sales.SalesOrderHeader soh


In [ ]:
select p.BusinessEntityID
    ,p.PersonType
    ,CONCAT_WS(' ',p.FirstName,p.MiddleName,p.LastName) Employee
    ,p.Suffix
    ,p.EmailPromotion
    ,ct.Name
from Person.Person p 
JOIN Person.BusinessEntityContact bec 
    on bec.PersonID = p.BusinessEntityID
JOIN Person.ContactType ct 
    on ct.ContactTypeID = bec.ContactTypeID


In [ ]:
select p.BusinessEntityID
    ,p.PersonType
    ,CONCAT_WS(' ',p.FirstName,p.MiddleName,p.LastName) Employee
    ,p.Suffix
    ,p.EmailPromotion
from Person.Person p 
JOIN HumanResources.Employee e 
    on e.BusinessEntityID = p.BusinessEntityID


In [ ]:
select p.BusinessEntityID
    ,p.PersonType
    ,CONCAT_WS(' ',p.FirstName,p.MiddleName,p.LastName) Employee
    ,p.Suffix
    ,p.EmailPromotion
from Person.Person p 
JOIN Person.BusinessEntity be 
    on be.BusinessEntityID = p.BusinessEntityID